In [1]:
%matplotlib qt5

from fmcal import fm_cal
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
import time

Example 2: Calibrating to a vairabl Cd. The iRIC FaSTMECH (FM) project is located in the folder test/test_var_cd. Note that the Geographic Data | roughness has 2 converage polygons with values 0 and 1. 

In your own project make sure that these values have been mapped to the grid (ie. From the menu Grid->Attributes Mapping->Execute. Also that the Calculations Conditions have been specified and saved. The following example illustrates how to vary the value of roughness over a range specified in the .ini file and determine a calibrated roughness for the reach.

Note: For your own project you should spend some time calibrating the project in the iRIC application before launching this code to know that you have a good set of calculation conditions, boundary conditions, and some bounds as to what the roughness is.

In [2]:
%%capture cal2_output
# magic command to capture output of model output in calibration loop.  
#  Output is displayed in next cell command
# comment line below if you want to see in real-time.  

from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import time

cal2 = fm_cal('../test/test_varcd_config.ini')
cal2.initialize()

fig = plt.figure()
ax = fig.add_subplot(111)
cax = make_axes_locatable(ax).append_axes('right', size='5%', pad='2%')
plt.ion()

fig.show()
fig.canvas.draw()
x = np.arange(cal2.mcdmin['0'], cal2.mcdmax['0'], cal2.mcdinc['0'])
numx = len(x)
y = np.arange(cal2.mcdmin['1'], cal2.mcdmax['1'], cal2.mcdinc['1'])
numy = len(y)
X, Y = np.meshgrid(x, y, indexing='ij')
tcd ={}
tcdind = {}
tcount = 0
tmpz = np.zeros(shape=(numx,numy))
for cdind0, cd0 in enumerate(x):
    tcd[0] = cd0
    tcdind[0] = cdind0
    for cdind1, cd1 in enumerate(y):
        tcd[1] = cd1
        tcdind[1] = cdind1
        numind = len(tcdind)
        # simres = cal.update_var(cdind0, cd0, cdind1, cd1)
        simres = cal2.update_var(tcount, tcd)
        tmpz[cdind0,cdind1] = simres.loc[tcount, 'rmse']
        # tmpz =np.reshape(simres['rmse'].values,(numx,numy))
        cs = ax.contourf(X, Y, tmpz, 20, cmap=plt.cm.bone_r)
        ax.set_xlabel('cd0')
        ax.set_ylabel('cd1')
        cbar = fig.colorbar(cs,cax=cax)
        cbar.ax.set_ylabel('RMSE')
        #     ax.plot(simres[:cdind,0], simres[:cdind,1], '-o')  # fit the line
        fig.canvas.update()
        fig.canvas.flush_events()
        plt.pause(1)
        plt.draw()
        tcount+=1
#     fig.canvas.draw()   # draw
#     time.sleep(0.5)    #sleep
plt.savefig('testfig_varcd.png', dpi=300)
cal2.resdf.to_csv(cal2.rmse_file)
tmp = 0

In [2]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Spectral11
output_notebook()
file = '../test/cal_var_cd/test_var_rmse.csv'
cald = pd.read_csv(file)
cal2 = fm_cal('../test/test_varcd_config.ini')
cal2.initialize()
cald.head()

x = np.arange(cal2.mcdmin['0'], cal2.mcdmax['0'], cal2.mcdinc['0'])
numx = len(x)
y = np.arange(cal2.mcdmin['1'], cal2.mcdmax['1'], cal2.mcdinc['1'])
numy = len(y)
p = figure(x_range=(min(cald['cd0'].values), max(cald['cd0'].values)), 
           y_range=(min(cald['cd0'].values), max(cald['cd0'].values)))
#            plot_width=800, plot_height=800)
p.image(image=cald['rmse'].values, x=0,y=0, dw=numx, dh=numy, \
        palette='Spectral11')

# p.title.text='Green River Variable Cd Calibraion'
# p.xaxis.axis_label='Drag Coefficient Polygon 0'
# p.yaxis.axis_label='Drag Coefficient Polygon 1'
# hover = HoverTool()
# hover.tooltips=[
#     ('Cd0', '@cd0'),
#     ('RMSE', '@rmse'),
#     ('Discharge', '@Discharge')
# ]

# p.add_tools(hover)
# output_file('contour.html',title='Contour Plot')
show(p)

Loading BokehJS ...

In [3]:
show(p)

In [4]:
cald.head()


,Unnamed: 0,index,cd0,cd1,Discharge,rmse
0,0,0.0,0.006,0.0085,241.0,0.054845
1,1,1.0,0.006,0.0087,241.0,0.053749
2,2,2.0,0.006,0.0089,241.0,0.052733
3,3,3.0,0.006,0.0091,241.0,0.051868
4,4,4.0,0.006,0.0093,241.0,0.051137
